# Polinomial Interpolation, Splines and Minimum Squares Approximation

## Polinomial Interpolation - Lagrange Method

#### Defining the polynomial

Given a set of $n+1$ points $(x_i, y_i)$, we can form a polynomial $p_n$ of degree at most $n$ defined by:

$ p_n(x) = l_0(x) f_0 + l_1(x) f_1 + ... + l_n(x) f_n $

In other words:

$ p_n(x) = \Sigma_{k=0}^{n}{l_k(x)f_k}$

Definition of $ l_k(x) $:

$ l_k(x) = \frac{(x-x_0)...(x-x_{k-1})(x-x_{k+1})...(x-x_n)}{(x_k-x_0)...(x_k-x_{k-1})(x_k-x_{k+1})...(x_k-x_n)} \Leftrightarrow$

$ l_k(x) = \Pi_{i=0}^{n}{\frac{x-x_i}{x_k-x_i}} $ and $i \ne k$

#### Error in Polynomial Interpolation

We can describe the error of the approximation of the polynomial with the following expression:

$ |E_n(x)| = |f(x) - p_n(x)| \le \frac{M}{(n+1)!} \pi_{n+1}(x) $ 

where $ M = \max{|f^{(n+1)}(x)|} $ for $ x \in [a,b] $

and where $ \pi_{n+1}(x) = (x-x_0)(x-x_1)...(x-x_n) $

`This value will have to be calculated analytically.`

### Implementation of Polynomial Interpolation by the Lagrange Method

In [1]:
# x -> the value which we're going to calculate the approximation of
# x_vals -> list of x values available/given
# f_vals -> list of f values such that f_vals[i] = f(x_vals[i]), also given

def interpolation(x, x_vals, f_vals):
    result = 0
    for k in range(len(x_vals)):
        # calculate the value of l_k
        l_k = 1
        for i in range(len(x_vals)):
            if (i == k):
                continue
            l_k *= (x - x_vals[i]) / (x_vals[k] - x_vals[i])
        # calculate l_k * f_k
        result += l_k * f_vals[k]
    return result

### What if we are given a list with more than $ n+1 $ points?

We can only use $ n+1 $ points to make an approximation on a $n$-degree polynomial, so, how do we choose them?

We would like to have the most equidistant points to the $x$ value we are using.

In [2]:
# degree -> the degree we want the polinomial to have (it also indicates us the size of lists)

def format_lists(x, x_vals, y_vals, degree):
    if (len(x_vals) == degree+1):
        return x_vals, y_vals
    i0 = 0 # variables to store the indexes that bound the intervals
    i1 = 0
    d = degree
    # determine the two closest indexes such that x_vals[i0] <= x <= x_vals[i1]
    for i in range(1, len(x_vals)):
        if (x_vals[i-1] <= x and x <= x_vals[i]):
            i0 = i-1
            i1 = i
            d -= 1
            break
    # expand the indexes that bound the list by choosing the values that are closer to the x value
    while (d > 0):
        # we can only expand to the right
        if (i0 == 0): i1 += 1
        # we can only expand to the left
        elif (i1 == len(x_vals)-1): i0 -= 1
        # expand such that it contains the next values closer to x
        else:
            if (abs(x_vals[i0-1]-x) > abs(x_vals[i1+1]-x)): i1 += 1
            else: i0 -= 1
        d -= 1
    # slice the lists to the obtained boundaries
    x_vals = x_vals[i0:i1+1]
    y_vals = y_vals[i0:i1+1]
    return x_vals, y_vals


### New version of the Implementation of Polynomial Interpolation by the Lagrange Method

In [3]:
def format_lists(x, x_vals, y_vals, degree):
    if (len(x_vals) == degree+1):
        return x_vals, y_vals
    i0 = 0 # variables to store the indexes that bound the intervals
    i1 = 0
    d = degree
    # determine the two closest indexes such that x_vals[i0] <= x <= x_vals[i1]
    for i in range(1, len(x_vals)):
        if (x_vals[i-1] <= x and x <= x_vals[i]):
            i0 = i-1
            i1 = i
            d -= 1
            break
    # expand the indexes that bound the list by choosing the values that are closer to the x value
    while (d > 0):
        # we can only expand to the right
        if (i0 == 0): i1 += 1
        # we can only expand to the left
        elif (i1 == len(x_vals)-1): i0 -= 1
        # expandir de forma a conter o x_i mais proximo de x
        # expand such that it contains the next values closer to x
        else:
            if (abs(x_vals[i0-1]-x) > abs(x_vals[i1+1]-x)): i1 += 1
            else: i0 -= 1
        d -= 1
    # slice the lists to the obtained boundaries
    x_vals = x_vals[i0:i1+1]
    y_vals = y_vals[i0:i1+1]
    return x_vals, y_vals

def interpolation(x, x_vals, f_vals, degree):
    x_vals, f_vals = format_lists(x, x_vals, f_vals, degree)
    result = 0
    for k in range(len(x_vals)):
        # calculate the value of l_k
        l_k = 1
        for i in range(len(x_vals)):
            if (i == k):
                continue
            l_k *= (x - x_vals[i]) / (x_vals[k] - x_vals[i])
        # calculate l_k * f_k
        result += l_k * f_vals[k]
    return result

## Newton's Divided Differences

### Definition of a Newton's Divided Difference

We define the value of a divided difference of order $n$ by:

$ f[x_0, ..., x_n] = (f[x_1, ..., x_n] - f[x_0, ..., x_{n+1}]) / (x_n - x_0) $

A more general definition of a divided difference of order $j$, starting at $x_i$: 

$ f[x_i, ..., x_{i+j}] = (f[x_{i+1}, ..., x_{i+j}] - f[x_i, ..., x_{i+j-1}]) / (x_{i+j} - x_i) $

We can create a table with columns that contain the values for certain degree divided differences.

### Implementation of Divided Differences Table Computation

In [4]:
# X: list of x values
# F: list of correspondent x values passed to the function f

def divided_differences_table(X, F):
    table = [[0 for _ in range(len(X))] for _ in range(len(X))] # left-most columns filled with the f values
    for j in range(len(F)):
        for i in range(len(F)-1-j, -1, -1):
            if j == 0:
                table[i][j] = F[i]
            else:
                table[i][j] = (table[i+1][j-1] - table[i][j-1]) / (X[i+j] - X[i])
    return table

### Getting the approximation value using Newton's Divided Differences

After getting the table, we want to compute the approximation for the value we're trying to calculate.

To perform that calculation, we use the approximation:

$ p_n(x) = f(x_0) + (x-x_0) f[x_0, x_1] + ... + (x-x_0)(x-x_1)...(x-x_{n-1})f[x_0, x_1, ..., x_n] $

Which is equivalent to:

$ p_n(x) = f(x_0) + \sum_{j=1}^{n}{\pi_{j}(x) f[x_0, ..., x_j]} $

In [5]:
def divided_differences_approximation(x, X, F):
    table = divided_differences_table(X, F)
    result = table[0][0]
    for j in range(1, len(X)):
        # calculate pi_i(x)
        pi_i = 1
        for k in range(j):
            pi_i *= x-X[k]
        # sum pi_i(x) * f[x_0, ..., x_i]
        result += pi_i * table[0][j]
    return result

Testing with the theoretical class example:

In [6]:
X = [2.1, 2.2, 2.3, 2.4, 2.5]
F = [0.32222, 0.34242, 0.36173, 0.38021, 0.39794]

x = 2.15

print(f"{divided_differences_approximation(x, X, F): .5f}")

 0.33243


## Splines

### Linear Splines

Polynomial interpolation by splines divides the domain/interval of approximation into segments and produces an approximated function for each segment.

In particular, Linear Splines compute **linear polynomials** to approximate the exact function in each segment.

Let $ S_i $ be the spline function that approximates the function at the $i$-th interval $(x_{i-1} \le x \le x_i)$.

Then, these splines must follow $ S_i(x_{i-1}) = f_{i-1} $ and $ S_i(x_i) = f_i $

We can define the spline for the $i$-th interval by:

$ S_i(x) = \frac{x-x_{i-1}}{h_i} f_i - \frac{x-x_i}{h_i} f_{i-1} $ with $ h_i = x_i - x_{i-1} $

The majorant of the error of an approximation by Linear Splines can be calculated by:

$ |f(x)-S(x)| \le \frac{1}{8}Mh^2 $

where

- $ M = \max_{x\in[a,b]}{|f''(x)|} $
- $ h = \max{h_i} $

Should we compute the splines for each interval if we want to find the approximated value for a point $x$?

No! We only need to identify which interval contains $x$, and calculate the value for that specific $x$ value.

### Linear Splines - Implementation

In [7]:
def spline_linear(x, X, F):
    # determine the values that bound x, and save their indexes
    i0 = 0
    i1 = 0
    for i in range(1, len(X)):
        if X[i-1] <= x and x <= X[i]:
            i0 = i-1
            i1 = i
            break

    # calculate the approximation by linear interpolation
    l_0 = (x - X[i1]) / (X[i1] - X[i0])
    l_1 = (x - X[i0]) / (X[i1] - X[i0])
    return l_1 * F[i1] - l_0 * F[i0]

Example:

In [8]:
X = [1, 2, 3, 4]
F = [1, 1/2, 1/3, 1/4]

print(spline_linear(1.5, X, F))

0.75


### Cubic Splines

Essentially, it follows the same idea as the linear splines.

Considering $n$ intervals bounded by $n+1$ x values, we will define a Spline $S_i$ with a (at most) **cubic polynomial** for each interval.

A cubic spline $S_i$ is defined as

$ S_i(x) = M_{i-1} \frac{(x_i - x)^3}{6 h_i} + M_i \frac{(x-x_{i-1})^3}{6 h_i} + (f_{i-1} - M_{i-1} \frac{h_i^2}{6}) \frac{x_i - x}{h_i} + (f_i - M_i \frac{h_i^2}{6}) \frac{x - x_{i-1}}{h_i} $

Therefore, to construct a spline, we need to find the $M$ values.

These can be computed by solving the system of $n+1$ equations, where $n-1$ of them are in the form

$ \frac{h_i}{6} M_{i-1} + \frac{h_i + h_{i+1}}{3} M_i + \frac{h_{i+1}}{6} M_{i+1} = \frac{f_{i+1} - f_i}{h_{i+1}} - \frac{f_i - f_{i-1}}{h_i} $

with $ i = 1, ..., n-1 $.

The last 2 equations of the system depend on the type of the Spline:

- Complete Cubic Spline  
  $ \frac{f_1 - f_0}{h_1} - \frac{h_1}{6} M_1 - \frac{h_1}{3} M_0 = f'_0 $  
  $ \frac{f_n - f_{n-1}}{h_n} - \frac{h_n}{6} M_{n-1} - \frac{h_n}{3} M_n = f'_n $

- Natural Cubic Spline  
  $ M_0 = 0 $  
  $ M_n = 0 $

### Error on the approximation using Cubic Splines

The error can be calculated by

$ |f(x) - S(x)| \le \frac{5}{384} M h^4 $

where 
- $ M = \max_{x\in[a,b]}{|f^{(4)}(x)|} $
- $ h = \max{h_i} $

## Minimum Squares Method

### Introduction

Let $ p_n(x) = \Sigma_{k=0}^{n}{a_k x^k} $ be the polynomial that approximates a set of $N+1$ points $ (x_i, f_i)_{i = 0, ..., N} $

The residue for $x_i$ is $ R_i = f_i - p_n(x_i) $, and, therefore, we can define the residue vector as:

$ R = \begin{pmatrix} R_0 \\ R_1 \\ \vdots \\ R_N \end{pmatrix} $

The Minimum Squares Method aims to find the coeficients $a_0$, $a_1$, ..., $a_n$ that minimizes $ ||R||^2 $.

Let &nbsp;&nbsp;&nbsp;
$ X = \begin{pmatrix} 1 \\ x \\ x^2 \\ \vdots \\ x^n \end{pmatrix} $
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
$ X_i = \begin{pmatrix} 1 \\ x_i \\ x_i^2 \\ \vdots \\ x_i^n \end{pmatrix} $
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
$ a = \begin{pmatrix} a_0 \\ a_1 \\ \vdots \\ a_n \end{pmatrix} $

**Note:** The polynomial $ p_n(x) $ can be defined as $ p_n(x) = X^T a $

Solving the system composed by the following $N+1$ equations of type 

$ R_i = f_i - X_i^T a$

we can find the vector $a$ that minimizes $||R||^2$.

Let $ A = \begin{pmatrix} 1 & 1 & ... & 1 \\ x_0 & x_1 & ... & x_N \\ x_0^2 & x_1^2 & ... & x_N^2 \\ \vdots & \vdots &  & \vdots \\ x_0^n & x_1^n & ... & x_N^n \end{pmatrix}_{A \in \R^{(n+1)\times(N+1)}}$ 

The residue vector can be re-defined as $ R = f - A^T a $ and the system defined above can be re-written as 
$ A R = 0 \Leftrightarrow (A A^T) a = A f $.

We aim to obtain the values of $a$.

### Implementation of the Minimum Squares Method

In [9]:
import numpy as np

# X: list of x values
# F: list of f values
# n: the degree of the polinomial
# returns a list with the coeficients "a" = [a_0, a_1, ..., a_n] 
def MinimumSquares(X, F, n):
    # define matrix A
    A = [[x**k for x in X] for k in range(n+1)]
    A = np.array(A)

    # define the transpose matrix of A, A^T
    At = np.transpose(A)

    # define matrix f
    f = np.array(F)

    # to solve the system of equations defined by A*At*a = A*f in order to a
    # we can use numpy.linalg.solve() function
    # x = numpy.linalg.solve(A, B) solves the system of equations represented by A*x = B in order to x
    a = np.linalg.solve(np.matmul(A, At), np.matmul(A, f))
    return a

# coefs: list of the polinomial coefficients in a_0, a_1, ..., a_n order
# x: x value to calculate the p_n(x) of
def polynomial(coefs, x):
    res = 0
    for i in range(len(coefs)):
        res += coefs[i] * x**i
    return res

# returns the square of the norm of the residue vector
def residue_norm_squared(X, F, coefs):
    residue = 0
    for i in range(len(X)):
        residue += (F[i] - polynomial(coefs, X[i]))**2
    return residue


### Minimum Squares Method with weights

We can assign a weight $w_i$ to each observation $(x_i, f_i)$ in the sense that we "trusting" more or less that point.

In that case, the system of equations to find the coefficients that best approximate the given points is defined as

$ A W A^T a = A W f $

where $ W = \begin{pmatrix} w_0 &  &  & 0 \\  & w_1 &  &  \\  &  & \ddots &  \\ 0 &  &  & w_N \end{pmatrix} $

### Implementation of the Minimum Squares Method with weights

In [10]:
import numpy as np

# X: list of x values
# F: list of f values
# W: list of the weights
# n: the degree of the polinomial
# returns a list with the coeficients "a" = [a_0, a_1, ..., a_n] 
def MinimumSquares(X, F, W, n):
    # define matrix W
    W = np.diag(W)

    # define matrix A
    A = [[x**k for x in X] for k in range(n+1)]
    A = np.array(A)

    # define the transpose matrix of A, A^T
    At = np.transpose(A)

    # define matrix f
    f = np.array(F)

    # solve A*W*At*a = A*W*f
    AWAt = np.matmul(np.matmul(A, W), At)
    AWf = np.matmul(np.matmul(A, W), f)
    a = np.linalg.solve(AWAt, AWf)
    return a


### Generalization of approximation in the direction of the Minimum Squares

We might want to use any kind of combination of non-polynomial functions.

We can use the Minimum Squares Method to find the coefficients to construct a linear combination a given set of functions $\phi_{0}(x), \phi_{1}(x), ..., \phi_{n}(x)$.

The difference to the system of equations of the weighted Minimum Squares Method, is the definition of the $A$ matrix, which, in this case, is

$ A = \begin{pmatrix} \phi_{0}(x_0) & \phi_{0}(x_1) & ... & \phi_{0}(x_N) \\ \phi_{1}(x_0) & \phi_{1}(x_1) & ... & \phi_{1}(x_N) \\ \vdots & \vdots &  & \vdots \\ \phi_{n}(x_0) & \phi_{n}(x_1) & ... & \phi_{n}(x_N) \end{pmatrix} $

Notebook criado por António Cardoso em 2023 para a Unidade Curricular "Métodos Numéricos (M2039)" na Faculdade de Ciências da Universidade do Porto.

$_{meow}$